In [1]:
from keras.datasets import boston_housing

Using TensorFlow backend.


In [2]:
(train_data, train_targets),(test_data, test_targets) = boston_housing.load_data()

In [3]:
mean = train_data.mean(axis=0)
train_data = -mean
std = train_data.std(axis=0)
train_data /= std

In [4]:
test_data -= mean #왜 test로 계산하지 않고 train으로 계산하는지?
test_data /= std 

In [5]:
from keras import models
from keras import layers

In [18]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop',
                 loss='mse',
                 metrics=['mae'])
    return model

In [19]:
import numpy as np

In [20]:
k=4
num_val_samples = len(train_data) // 4
num_epochs = 5
all_scores = []

for i in range(k):
    print('처리중인 폴드 #',i+1)
    val_data = train_data[i*num_val_samples:(i+1)*num_val_samples] #검증 데이터 준비
    val_targets = train_targets[i*num_val_samples:(i+1)*num_val_samples]
    
    partial_train_data = np.concatenate([train_data[:i*num_val_samples], #훈련 데이터 준비, 다른 분할 전체
                                           train_data[(i+1)*num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i*num_val_samples],
                                           train_targets[(i+1)*num_val_samples:]], axis=0)
    
    model = build_model() #모델 구성
    model.fit(partial_train_data, partial_train_targets,
             epochs = num_epochs, batch_size=1, verbose=0) #verbose = 0 훈련과정 출력x
    
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0) #모델 평가
    all_scores.append(val_mae)

처리중인 폴드 # 1


IndexError: tuple index out of range